In [ ]:
### change this to your newly created output bucket folder ###
bucket= "output-bucket-default-i5frrop8"

import pyspark

In [ ]:

patients = spark.read.parquet("s3a://%s/patients/_raw" %(bucket))

print(patients.count())
patients.printSchema()


In [ ]:

from pyspark.sql import functions as f
from pyspark.sql import types as t
from pyspark.sql import Window as w
from pyspark.sql.functions import udf

def ascii_ignore(x):
    return x.encode('ascii', 'ignore').decode('ascii')

ascii_udf = udf(ascii_ignore)

def trim_to_null(c):
  return (
    f.lower(
      f.when(f.trim(f.col(c)) == '', None)
      .when(f.trim(f.col(c)) == 'null', None)
      .otherwise(f.trim(f.col(c)))
    )
  )

keep_cols = ['id', 'birth_date', 'first_name', 'surname', 'social_sec_num','suffix']

people_bronze = patients.select(*keep_cols)
people_bronze = people_bronze.withColumn("first_name", ascii_udf('first_name'))
people_bronze = people_bronze.withColumn("last_name", ascii_udf('surname'))
people_bronze = people_bronze.withColumn("suffix", f.lower(trim_to_null("suffix")))
people_bronze = people_bronze.drop(people_bronze.surname)
people_bronze.printSchema()
people_bronze.write.format("parquet").mode("overwrite").save("s3a://%s/patients/_bronze" %(bucket))

In [ ]:
people_bronze.write.mode("overwrite").saveAsTable("bronze_patients")

In [ ]:

distinct_id = spark.sql("SELECT distinct id FROM bronze_patients").count()
print(distinct_id)

distinct_all = spark.sql("SELECT id, first_name, last_name, birth_date, social_sec_num FROM bronze_patients").count()
print(distinct_all)

distinct_ssn = spark.sql("SELECT distinct social_sec_num FROM bronze_patients").count()
print(distinct_ssn)

In [ ]:
parquet = spark.read.parquet("s3a://%s/patients/_clean/parquet" %(bucket))
print(parquet.count())
parquet.printSchema()

delta = spark.read.format("delta").load("s3a://%s/patients/_clean/delta" %(bucket))
print(delta.count())
delta.printSchema()

In [ ]:
parquet.write.mode("overwrite").saveAsTable("patients_clean")


In [ ]:

spark.sql("select * from patients_clean").toPandas()